# Bootstrapping Labels using GPT-4
This is a quick notebook to show how you can use the OpenAI API to bootstrap labels for your data. In this case, we'll be labeling sentiment examples. Before you begin, you will need an [OpenAI account](https://platform.openai.com/) to obtain an API key.

In [1]:
# Install OpenAI Python client
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
import openai
import re

In [5]:
import configparser

config = configparser.ConfigParser()
config.read('/content/drive/MyDrive/openapi.txt')
openai.api_key = config['global']['OPENAI_API_KEY']

This is our function to create a prompt and request for the GPT models to respond to. Note we are using GPT-3.5-turbo here, but we can replace that model with GPT-4 as well.

In [6]:
def get_sentiment(input_text):
    prompt = f"Respond in the json format: {{'response': sentiment_classification}}\nText: {input_text}\nSentiment (positive, neutral, negative):"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=40,
        n=1,
        stop=None,
        temperature=0.5,
    )
    response_text =  response.choices[0].message['content'].strip()
    sentiment = re.search("negative|neutral|positive", response_text).group(0)
    # Add input_text back in for the result
    return {"text": input_text, "response": sentiment}


In [7]:
# Test single example
sample_text = "I had a terrible time at the party last night!"
sentiment = get_sentiment(sample_text)
print("Result\n",f"{sentiment}")

Result
 {'text': 'I had a terrible time at the party last night!', 'response': 'negative'}


To verify this data, we'll use the following function to convert the results into the Label Studio format. Then we can load the data into Label Studio for human evaluation.

In [8]:
def convert_ls_format(input_dict):
    """
    Convert sentiment analysis output from a simple format to Label Studio's prediction format.

    Args:
        input_dict (dict): A dictionary containing text and response keys. Example:
            {
                "text": "I love going to the park on a sunny day.",
                "response": "positive"
            }

    Returns:
        dict: A dictionary in Label Studio's prediction format.
    """

    score_value = 1.00  # We don't know the model confidence
    output_dict = {
        "data": {
            "text": input_dict["text"]
        },
        "predictions": [
            {
                "result": [
                    {
                        "value": {
                            "choices": [
                                input_dict["response"].capitalize()
                            ]
                        },
                        "from_name": "sentiment",
                        "to_name": "text",
                        "type": "choices"
                    }
                ],
                "score": score_value,
                "model_version": "gpt-3.5-turbo"
            }
        ]
    }
    return output_dict

In [9]:
# Convert to Label Studio format
print(convert_ls_format(sentiment))

{'data': {'text': 'I had a terrible time at the party last night!'}, 'predictions': [{'result': [{'value': {'choices': ['Negative']}, 'from_name': 'sentiment', 'to_name': 'text', 'type': 'choices'}], 'score': 1.0, 'model_version': 'gpt-3.5-turbo'}]}


Putting it all together, we'll create a file with some examples in it (one per line) that we'll evaluate the sentiment on. Finally, we'll write all of our examples out to a file which can be imported by Label Studio.

In [10]:
%%writefile input_texts.txt
I love going to the park on a sunny day.
The customer service was terrible; they were rude and unhelpful.
I am neither happy nor sad about the new policy changes.
The cake was delicious and the presentation was fantastic.
I had a really bad experience with the product; it broke after two days.

Writing input_texts.txt


In [11]:
import json

input_file_path = "input_texts.txt"
output_file_path = "output_responses.json"

with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
    examples = []
    for line in input_file:
        text = line.strip()
        if text:
            examples.append(convert_ls_format(get_sentiment(text)))
    output_file.write(json.dumps(examples))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')